# Homework 8

## FINM 36700 - 2023

### UChicago Financial Mathematics

**Professor**
* Mark Hendricks
* hendricks@uchicago.edu

**Students**

Anya Zakharov azakharov@uchicago.edu

Gleb Skvortsov goskvortsov@uchicago.edu

Kai Wen Tay kaiwent@uchicago.edu

Ka Hang Toong khtoong@uchicago.edu


Tikhonov Sergei tikhonov@uchicago.edu


## Modules

In [1]:
import os
import pandas as pd
from scipy.stats import norm
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
from arch.univariate import GARCH
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

import matplotlib.pyplot as plt

## Helping functions

In [2]:
def performance_summary(return_data):

    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*12)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(12))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()

    
    return summary_stats

## Reading Data

In [5]:
portdf = pd.read_excel('ltcm_exhibits_data.xlsx', sheet_name=1, skiprows=(0,1), usecols='A:E').dropna().rename(columns={'Unnamed: 0':'Date','Gross Monthly Performancea':'GrossLTCM','Net Monthly Performanceb':'NetLTCM'}).set_index('Date')
portdf.index = portdf.index.to_period('M').to_timestamp('M')

rets = pd.read_excel('gmo_analysis_data.xlsx', sheet_name=2, usecols = "A:B").rename(columns={'Unnamed: 0': 'Date'}).set_index('Date').loc['1994-03':'1998-07']
portdf['SPY'] = rets['SPY']

rf = pd.read_excel('gmo_analysis_data.xlsx', sheet_name=3).rename(columns={'Unnamed: 0': 'Date'}).set_index('Date').loc['1994-03':'1998-07']
retsx = portdf.subtract(rf['US3M'],axis=0)

retsx.head()

,Fund Capital ($billions),GrossLTCM,NetLTCM,Index of Net Performance,SPY
Date,,,,,
1994-03-31,1.097033,-0.013967,-0.015967,0.987033,-0.044868
1994-04-30,1.096692,0.010692,0.004692,0.996692,0.007904
1994-05-31,1.196408,0.064408,0.049408,1.046408,0.012348
1994-06-30,1.196450,-0.042550,-0.032550,1.016450,-0.026429
1994-07-31,1.396342,0.112342,0.080342,1.096342,0.028668


## 2 LTCM Risk Decomposition

• On Canvas, find the data file, “ltcm exhibits data.xlsx”. Get the gross and net (total) returns
of LTCM from “Exhibit 2”.

• Get the returns on SPY as well as the risk-free rate from the file, “gmo analysis data”.

### 1 Summary stats.

(a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios. (Annualize them.)

In [16]:
mean_vol_performance = []
series = ['GrossLTCM', 'NetLTCM', 'SPY']

for elem in series:
    mean_vol_performance.append(performance_summary(portdf[[elem]])[['Mean', 'Volatility', 'Sharpe Ratio']])
pd.concat(mean_vol_performance)

,Mean,Volatility,Sharpe Ratio
GrossLTCM,0.293887,0.136354,2.155321
NetLTCM,0.207170,0.111904,1.851315
SPY,0.225633,0.112547,2.004777


(b) Report the skewness, kurtosis, and (historic) VaR(.05).

In [18]:
tail_performance = []
series = ['GrossLTCM', 'NetLTCM', 'SPY']

for elem in series:
    tail_performance.append(performance_summary(portdf[[elem]])[['Skewness', 'Excess Kurtosis', 'VaR (0.05)']])
pd.concat(tail_performance)

,Skewness,Excess Kurtosis,VaR (0.05)
GrossLTCM,-0.296428,1.569354,-0.026400
NetLTCM,-0.817870,2.905537,-0.022400
SPY,-0.436267,-0.370029,-0.042796


(c) Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

**Mean-vol returns**:

GrossLTCM has substantially larger mean excess returns and slightly higher volatility, hence higher Sharpe Ratio. However, NetLTCM has lower mean excess returns compared to SPY, and relatively same volatility. 

**Tail risk metrics**:

NetLTCM shows much more tail risk that SPY does. 

### 2 

Using the series of net LTCM excess returns, denoted $\tilde{r}^{\text{LTCM}}$, estimate the following regression:

$$
\tilde{r}^{\text{LTCM}} = \alpha + \beta^m \tilde{r}^{m} + \epsilon_t
$$

(a) Report $\alpha$ and $\beta^m$. Report the $R^2$ stat.

In [40]:
X = sm.tools.add_constant(portdf['SPY'])
y = portdf['NetLTCM']
model = sm.OLS(y, X).fit()

alpha = model.params[0]
beta = model.params[1]
RSquared = model.rsquared

pd.DataFrame({'Alpha': alpha * 12, 'Market beta': beta, 'R2': RSquared}, index = ['Stats'])

,Alpha,Market beta,R2
Stats,0.175588,0.139969,0.019817


(b) From this regression, does LTCM appear to be a “closet indexer”?

**Answer**: We obtained very small $R^2$ for pairwise regression. As a result, excess net returns of LTCM have small correlation with market returns, so we cannot conclude that LTCM appear to be a "closet indexer". This result is expected, because the strategy of this fund was believed to be market neutral

(c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium
we expect from market exposure?

**Answer**: LTCM indeed provides substantial excess returns beyond the market. 

3 Let’s check for non-linear market exposure. Run the following regression on LTCM’s net excess returns:

$$
\tilde{r}^{\text{LTCM}} = \alpha + \beta_1 \tilde{r}^{m} + \beta_2 (\tilde{r}^{m})^2 + \epsilon_t
$$

(a) Report $\beta_1$, $\beta_2$ and the $R^2$ stat.

In [48]:
X = sm.tools.add_constant(pd.concat([portdf['SPY'], portdf['SPY'] ** 2], axis = 1))
y = portdf['NetLTCM']
model = sm.OLS(y, X).fit()

alpha = model.params[0]
beta_1 = model.params[1]
beta_2 = model.params[2]
RSquared = model.rsquared

pd.DataFrame({'Alpha': alpha * 12, 'Beta 1': beta_1, 'Beta 2': beta_2, 'R2': RSquared}, index = ['Stats'])

,Alpha,Beta 1,Beta 2,R2
Stats,0.198894,0.189561,-2.069263,0.025977


(b) Does the quadratic market factor do much to increase the overall LTCM variation explained
by the market?

**Answer**: Even though significant negative Beta 2 indicates non-linear market exposure, we still observe small $R^2$ and hence quadratic market factor does not improve regression performance.

(c) From the regression evidence, does LTCM’s market exposure behave as if it is long market
options or short market options?

**Answer**: As was observed above, we obtained negative quadratic beta of market factor. It means that for large enough returns of market, the higher excess market returns are the lower excess LTCM returns. As a result, we can conclude that LTCM market exposure behave as if it is short market options.

(d) Should we describe LTCM as being positively or negatively exposed to market volatility?

**Answer**: We observed that LTCM market exposure behave as if it is short market options, we can conclude the fund negatively exposed to market volatility.

### 4

Let’s try to pinpoint the nature of LTCM’s nonlinear exposure. Does it come more from exposure
to up-markets or down-markets? Run the following regression on LTCM’s net excess returns:

$$
\tilde{r}^{\text{LTCM}} = \alpha + \beta \tilde{r}^m_t + \beta_u \max(\tilde{r}^m_t - k_1, 0) + \beta_d \max(k_2 - \tilde{r}^m_t, 0) + \epsilon_t
$$

where k1 = .03 and k2 = −.03. (This is roughly one standard deviation of $\tilde{r}^m$.)

In [54]:
k1 = .03
k2 = -.03

up_market = np.maximum(portdf['SPY'] - k1, 0)
down_market = np.maximum(k2 - portdf['SPY'], 0)

X = sm.tools.add_constant(pd.concat([portdf['SPY'], up_market, down_market], axis = 1))
y = portdf['NetLTCM']
model = sm.OLS(y, X).fit()

alpha = model.params[0]
beta = model.params[1]
beta_u = model.params[2]
beta_d = model.params[3]
RSquared = model.rsquared

pd.DataFrame({'Alpha': alpha * 12, 'Beta': beta, 'Beta u': beta_u, 
              'Beta d': beta_d, 'R2': RSquared}, index = ['Stats'])

,Alpha,Beta,Beta u,Beta d,R2
Stats,0.138157,0.493472,-0.760903,1.572423,0.054671


Is LTCM long or short the call-like factor? And the put-like factor?

**Answer**: According to Beta up and Beta down, we can conclude that LTCM short call-like factor and long put-like factor.

(c) Which factor moves LTCM more, the call-like factor, or the put-like factor?

**Answer**: the put-like factor moves excess returns of LTCM more than call-like factor does, since absolute value of Beta u is higher than that of Beta d.

(d) In the previous problem, you commented on whether LTCM is positively or negatively
exposed to market volatility. Using this current regression, does this volatility exposure
come more from being long the market’s upside? Short the market’s downside? Something
else?

**Answer**: negative exposure is driven by shorting market upside, because of large positive Beta d (put-like factor).

## 3 The FX Carry Trade

Find an Excel data file, “fx carry data.xlsx”. The file has two sets of data:

• Risk-free rates across 5 currencies, as measured by annualized 3-month LIBOR rates.

• Spot FX rates, as direct quotes to the USD. (Note that all currencies are quoted as USD per
the foreign currency.)

For use in the homework, note the following:

• For risk-free rate data, $r^{f, i}_{t, t+1}$, the rate is known and reported in the data at time $t$. Namely,
any given date $t$ in the data file is reporting both $S^i_t$ and $r^{f, i}_{t, t+1}$.

• The theory says to use log risk-free rates. You have the risk-free rate in levels: use the following
equation to convert them:

$$
r^{f, i}_{t, t+1} = ln(1 + r^{f, i}_{t, t+1})
$$

• The theory says to use log spot FX prices. You have the FX prices in levels, so directly take
their logarithims:

$$
s^i_t = ln(S^i_t)
$$

### 1 The Static Carry Trade

Define the log return of holding the foreign currency using log values of the risk-free rate and
log values of the FX rates:

$$
r^i_{t+1} = s^i_{t+1} - s^i_t + r^{f, i}_{t, t+1}
$$

Then the excess log return relative to USD, is expressed as

$$
\tilde{r}^i_{t+1} = s^i_{t+1} - s^i_{t} + r^{f, i}_{t, t+1} - r^{f, $}_{t, t+1}
$$

For each foreign currency, i, calculate the excess log return series, ˜rt+1. Report the following
stats, (based on the excess log returns.) Annualize them.

(a) mean

(b) volatility

(c) Sharpe ratio

In [62]:
fxspot = pd.read_excel('fx_carry_data.xlsx',sheet_name=2,index_col='DATE')
rf = pd.read_excel('fx_carry_data.xlsx',sheet_name=1,index_col='DATE')

logrf = np.log(1 + rf).shift()
logfxspot = np.log(fxspot)

In [64]:
mean_vol_performance = []
series = ['USUK', 'USEU', 'USSZ', 'USJP']

excess_log_fx = pd.DataFrame(index=logfxspot.index)
excess_log_fx['USUK'] = logfxspot.diff()['USUK'] + logrf['GBP1M'] - logrf['USD1M']
excess_log_fx['USEU'] = logfxspot.diff()['USEU'] + logrf['EUR1M'] - logrf['USD1M']
excess_log_fx['USSZ'] = logfxspot.diff()['USSZ'] + logrf['CHF1M'] - logrf['USD1M']
excess_log_fx['USJP'] = logfxspot.diff()['USJP'] + logrf['JPY1M'] - logrf['USD1M']
excess_log_fx = excess_log_fx.dropna()

for elem in series:
    mean_vol_performance.append(performance_summary(excess_log_fx[[elem]])[['Mean', 'Volatility', 'Sharpe Ratio']])
pd.concat(mean_vol_performance)

,Mean,Volatility,Sharpe Ratio
USUK,-0.003502,0.086303,-0.040574
USEU,-0.004351,0.094714,-0.045944
USSZ,0.004312,0.098757,0.043662
USJP,-0.017415,0.091492,-0.190342


What differences do you see across currencies?

**Answer**:

The only currency pair that indicates positive Sharpe Ratio is the pair of US dollar and CHF. Volatilities are comparable among all the currencies. The pair of US dollar and Japanese Yen has the lowest mean returns (4 times less than any other currency).

### 2 Implications for UIP:

(a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

**Answer**: We can see that all USUK, USEU, USSZ very small mean excess returns, the fact which corresponds to Uncovered Interest Parity (UIP). However, USJP has relatively high absolute mean excess returns, which can indicate contradiction to UIP.

(b) A long position in which foreign currency offered the best Sharpe ratio over the sample?

**Answer**: According to computations provided above, the long USD and short JPY position can provide 1.74% returns and result 0.19 Sharpe ratio

(c) Are there any foreign currencies for which a long position earned a negative excess return
(in USD) over the sample?

**Answer**: Since USUK, USEU, and USJP have all negative mean returns, it indicates that a long position earned negative excess returns. However, USSZ has a positive mean return.

### 3 Predicting FX

For each foreign currency, test whether interest-rate differentials can predict growth in the
foreign-exchange rate. Do this by estimating the following forecasting regression:

$$
s^i_{t+1} - s^i_t = \alpha^i + \beta^i (r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1}) + \epsilon^i_{t+1}
$$

where $r^{f, i}$ denotes the risk-free rate of currency $i$, and $s^i$ denotes the FX rate for currency $i$. Again, note that both $r^{f, $}_{t, t+1}$ and $s_t$ are determined at time $t$.

(a) Make a table with columns corresponding to a different currency regression. Report the
regression estimates $\alpha^i$ and $\beta^i$ in the first two rows. Report the $R^2$ stat in the third row.

In [86]:
diff = {'USUK': logrf['USD1M'] - logrf['GBP1M'], 'USEU': logrf['USD1M'] - logrf['EUR1M'],
 'USSZ': logrf['USD1M'] - logrf['CHF1M'], 'USJP': logrf['USD1M'] - logrf['JPY1M']}
X = pd.DataFrame(diff).dropna()
y = logfxspot.diff().dropna()

results = []

for pair in X.columns:
    X_pair = sm.tools.add_constant(X[pair])
    y_pair = y[pair]

    model = sm.OLS(y_pair, X_pair).fit()

    alpha = model.params[0]
    beta = model.params[1]
    RSquared = model.rsquared
    results.append(pd.DataFrame({'alpha': alpha * 12, 'beta': beta, 'R2': RSquared}, index=[pair]))
    
res = pd.concat(results).T
res

,USUK,USEU,USSZ,USJP
alpha,-0.005868,0.007033,0.043574,-0.005996
beta,0.485836,-1.256358,-1.646596,0.371473
R2,0.000382,0.002610,0.003948,0.000501


(b) Suppose the foreign risk-free rate increases relative to the US rate.

1 For which foreign currencies would we predict a relative strengthening of the USD in
the following period?

**Answer**: To answer this question, we can look at the signs of betas. 

$$
s^i_{t+1} - s^i_t = \alpha^i + \beta^i (r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1}) + \epsilon^i_{t+1}
$$

According to the task:

$$
r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1} < 0 
$$

Then to expect a relative strengthening of the USD in the following period we need positive beta, which is the case for UK and JP.

2 For which currencies would we predict relative weakening of the USD in the following

**Answer**: For now, we have:

According to the task:

$$
r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1} > 0 
$$

Then to expect a relative strengthening of the USD in the following period we need negative beta, which is the case for EU and SZ.

3 This FX predictability is strongest in the case of which foreign currency?

**Answer**: According to $R^2$, we can conclude that predictability is the strongest in the case of SZ.

### 4 The Dynamic Carry Trade
Use this to write $\mathbb{E}_t(\tilde{r}^i_{t+1})$ as a function of the interest-rate differential as well as $\alpha$ and $\beta$ from this FX regression.

$$
\mathbb{E}_t(s_{t+1} - s_t) = \alpha + \beta(r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1})
$$

Then use the definition of excess (log) returns on FX:

$$
\tilde{r}^{i}_{t+1} = s_{t+1} - s_t - (r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1})
$$

Rearranging, this implies the following forecast for excess log returns:

$$
\mathbb{E}_t(\tilde{r}^{i}_{t+1}) = \alpha + (\beta - 1)(r^{f, $}_{t, t+1} - r^{f, i}_{t, t+1})
$$

(a) Use your regression estimates from Problem 3 along with the formula above to calculate
the fraction of months for which the estimated FX risk premium positive. That is, for each
i, calculate how often in the time-series we have:

$$
\mathbb{E}_t(\tilde{r}^i_{t+1}) > 0
$$

In [93]:
USUK_pred = res['USUK']['alpha'] / 12 + (res['USUK']['beta'] - 1) * X['USUK']
USEU_pred = res['USEU']['alpha'] / 12 + (res['USEU']['beta'] - 1) * X['USEU']
USSZ_pred = res['USSZ']['alpha'] / 12 + (res['USSZ']['beta'] - 1) * X['USSZ']
USJP_pred = res['USJP']['alpha'] / 12 + (res['USJP']['beta'] - 1) * X['USJP']

print(f'Expected USUK > 0 : {round(sum(USUK_pred > 0) / len(USUK_pred), 2) * 100}%')
print(f'Expected USEU > 0 : {round(sum(USEU_pred > 0) / len(USEU_pred), 2) * 100}%')
print(f'Expected USSZ > 0 : {round(sum(USSZ_pred > 0) / len(USSZ_pred), 2) * 100}%')
print(f'Expected USJP > 0 : {round(sum(USJP_pred > 0) / len(USJP_pred), 2) * 100}%')

Expected USUK > 0 : 24.0%
Expected USEU > 0 : 50.0%
Expected USSZ > 0 : 63.0%
Expected USJP > 0 : 0.0%


(b) Which currencies most consistently have a positive FX risk premium? And for which
currencies does the FX risk premium most often go negative?

**Answer**: As can be seen from the formulas above, EU and SZ have most consistently positive FX risk premiums, while UK and JP risk premiums are mostly negative.

(c) Explain how we could use these conditional risk premia to improve the static carry trade
returns calculated in Problem 1.

**Answer**: the fund can improve the static carry trade returns by taking advantage of conditional risk premia: borrowing at the JPY's risk-free rate and then investing in the risk-free rate of the US Dollar. Considering our prediction of the USD gaining strength against the JPY, this strategy could yield a significant risk premium from the carry trade.